In [1]:
import numpy as np
import pandas as pd

import pymysql
import dotenv
import os
import pickle
import seaborn as sns
import matplotlib

from datetime import timedelta

import matplotlib.pyplot as plt

# Normalization/Standardization
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder

from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, Lasso
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Dropout, Conv1D, GRU
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.saving import save_model

# 경고 무시 코드 추가
import warnings
warnings.filterwarnings('ignore')

def pltconfig_default() :
  sns.reset_defaults()
  %matplotlib inline

pltconfig_default()

matplotlib.rcParams

matplotlib.rcParams['font.family']

current_font_list = matplotlib.rcParams['font.family']

font_path = 'C:\\Windows\\Fonts\\batang.ttc'

kfont = matplotlib.font_manager.FontProperties(fname=font_path).get_name()

matplotlib.rcParams['font.family'] = [kfont] + current_font_list

In [2]:
St_NotEncode_data = pd.read_pickle("StandardScalar_final_data")

print(St_NotEncode_data.shape)

(17422, 9)


In [3]:
# Feature와 Label 분리하기 (일조시간)
def Feature_Label(datafile) :
    X = datafile.iloc[:,[4]]
    y = datafile.iloc[:,-1]

    return X, y

print("StandardScaler Not Encode Data")
SNE_X, SNE_y = Feature_Label(St_NotEncode_data)
print(SNE_X.shape, SNE_y.shape)
SNE_X_train, SNE_X_test, SNE_y_train, SNE_y_test = train_test_split(SNE_X, SNE_y, test_size=0.2, random_state=10, shuffle=False)
print(SNE_X_train.shape, SNE_X_test.shape, SNE_y_train.shape, SNE_y_test.shape)
print("=======================================")


WINDOW_SIZE=3
BATCH_SIZE=28

StandardScaler Not Encode Data
(17422, 1) (17422,)
(13937, 1) (3485, 1) (13937,) (3485,)


In [4]:
# 위 내용을 함수 형태로 만들고 싶은 경우

def windowed_dataset(x, y, window_size, batch_size, shuffle) :
  ds_x = tf.data.Dataset.from_tensor_slices(x).window(window_size, stride=1, shift=1, drop_remainder=True) 
  ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
  
  ds_y = tf.data.Dataset.from_tensor_slices(y[window_size:])
  
  ds = tf.data.Dataset.zip((ds_x, ds_y))
  
  if shuffle:
    ds = ds.shuffle(1000)
  
  return ds.batch(batch_size).prefetch(1)

In [5]:
train_data = windowed_dataset(SNE_X_train, SNE_y_train, WINDOW_SIZE, BATCH_SIZE, True)
train_data_ns = windowed_dataset(SNE_X_train, SNE_y_train, WINDOW_SIZE, BATCH_SIZE, False)
test_data = windowed_dataset(SNE_X_test, SNE_y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [6]:
for data in train_data.take(1) :
  print(f'{data[0].shape}, {data[1].shape}')

(28, 3, 1), (28,)


In [7]:
lstm_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal',
         activation='relu',
         input_shape=[WINDOW_SIZE, 1]),
  LSTM(128, activation='tanh', return_sequences=True),  
  LSTM(64, activation='tanh', return_sequences=True),
  LSTM(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [8]:
rnn_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 1]),
  SimpleRNN(128, activation='tanh', return_sequences=True),  
  SimpleRNN(64, activation='tanh', return_sequences=True),
  SimpleRNN(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [9]:
gru_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 1]),
  GRU(128, activation='tanh', return_sequences=True),  
  GRU(64, activation='tanh', return_sequences=True),
  GRU(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [10]:
loss = Huber() 
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss=loss, optimizer=optimizer, metrics=['mse', 'mae'])
rnn_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
gru_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [11]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
       
es = EarlyStopping(monitor='val_loss', patience=20)

mc_lstm = ModelCheckpoint('new_multi_stne_lstm_weight_c1.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_lstm_ns = ModelCheckpoint('new_multi_stne_lstm_weight_ns_c1.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn = ModelCheckpoint('new_multi_stne_rnn_weight_c1.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn_ns = ModelCheckpoint('new_multi_stne_rnn_weight_ns_c1.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru = ModelCheckpoint('new_multi_stne_gru_weight_c1.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru_ns = ModelCheckpoint('new_multi_stne_gru_weight_ns_c1.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)

In [12]:
lstm_history = lstm_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm])
lstm_history_ns = lstm_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm_ns])

Epoch 1/500


    497/Unknown - 14s 7ms/step - loss: 7614.8755 - mse: 183358000.0000 - mae: 7615.2715
Epoch 1: val_loss improved from inf to 7097.84619, saving model to new_multi_stne_lstm_weight_c1.h5
498/498 [==============================] - 19s 18ms/step - loss: 7616.2876 - mse: 183374352.0000 - mae: 7616.6836 - val_loss: 7097.8462 - val_mse: 170910672.0000 - val_mae: 7098.2729 - lr: 0.0010
Epoch 2/500
495/498 [============================>.] - ETA: 0s - loss: 7552.7285 - mse: 180925168.0000 - mae: 7553.1099
Epoch 2: val_loss improved from 7097.84619 to 7035.25195, saving model to new_multi_stne_lstm_weight_c1.h5
498/498 [==============================] - 4s 8ms/step - loss: 7560.4473 - mse: 181012752.0000 - mae: 7560.8296 - val_loss: 7035.2520 - val_mse: 167947104.0000 - val_mae: 7035.7202 - lr: 0.0010
Epoch 3/500
496/498 [============================>.] - ETA: 0s - loss: 7462.1250 - mse: 176890720.0000 - mae: 7462.5098
Epoch 3: val_loss improved from 7035.25195 to 6933.33887, saving model to n

In [13]:
rnn_history = rnn_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn])
rnn_history_ns = rnn_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn_ns])

Epoch 1/500


    486/Unknown - 5s 4ms/step - loss: 7633.3105 - mse: 184573648.0000 - mae: 7633.6924
Epoch 1: val_loss improved from inf to 7117.62939, saving model to new_multi_stne_rnn_weight_c1.h5
498/498 [==============================] - 7s 6ms/step - loss: 7624.3828 - mse: 183766480.0000 - mae: 7624.7661 - val_loss: 7117.6294 - val_mse: 171725792.0000 - val_mae: 7118.0562 - lr: 0.0010
Epoch 2/500
490/498 [============================>.] - ETA: 0s - loss: 7592.7051 - mse: 182591104.0000 - mae: 7593.0952
Epoch 2: val_loss improved from 7117.62939 to 7094.01660, saving model to new_multi_stne_rnn_weight_c1.h5
498/498 [==============================] - 2s 3ms/step - loss: 7601.3877 - mse: 182695536.0000 - mae: 7601.7788 - val_loss: 7094.0166 - val_mse: 170415072.0000 - val_mae: 7094.4849 - lr: 0.0010
Epoch 3/500
492/498 [============================>.] - ETA: 0s - loss: 7560.8662 - mse: 180823152.0000 - mae: 7561.2607
Epoch 3: val_loss improved from 7094.01660 to 7047.12744, saving model to new_mu

In [14]:
gru_history = gru_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru])
gru_history_ns = gru_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru_ns])

Epoch 1/500


    498/Unknown - 14s 7ms/step - loss: 7614.9609 - mse: 183370800.0000 - mae: 7615.3311
Epoch 1: val_loss improved from inf to 7095.79785, saving model to new_multi_stne_gru_weight_c1.h5
498/498 [==============================] - 16s 12ms/step - loss: 7614.9609 - mse: 183370800.0000 - mae: 7615.3311 - val_loss: 7095.7979 - val_mse: 170886832.0000 - val_mae: 7096.2158 - lr: 0.0010
Epoch 2/500
493/498 [============================>.] - ETA: 0s - loss: 7540.6865 - mse: 180545152.0000 - mae: 7541.0757
Epoch 2: val_loss improved from 7095.79785 to 7012.77490, saving model to new_multi_stne_gru_weight_c1.h5
498/498 [==============================] - 4s 8ms/step - loss: 7552.8262 - mse: 180751408.0000 - mae: 7553.2158 - val_loss: 7012.7749 - val_mse: 167291744.0000 - val_mae: 7013.2676 - lr: 0.0010
Epoch 3/500
496/498 [============================>.] - ETA: 0s - loss: 7429.5776 - mse: 175548672.0000 - mae: 7429.9722
Epoch 3: val_loss improved from 7012.77490 to 6897.45898, saving model to new

In [15]:
# 종합 결과

history_list = ["lstm_history", "rnn_history", "gru_history", "lstm_history_ns", "rnn_history_ns", "gru_history_ns"]
def result(historys) :
  for name, history in globals().items() :
    if name in history_list :
      print(f"-------------{name}-------------")
      val_loss = min(history.history['val_loss'])
      val_mse = min(history.history['val_mse'])
      val_mae = min(history.history['val_mae'])
      print(f"{name} Validation Loss:", val_loss)
      print(f"{name} Validation MSE:", val_mse)
      print(f"{name} Validation MAE:", val_mae)

result(history_list)

-------------lstm_history-------------
lstm_history Validation Loss: 3160.474365234375
lstm_history Validation MSE: 37370336.0
lstm_history Validation MAE: 3160.89453125
-------------lstm_history_ns-------------
lstm_history_ns Validation Loss: 3182.106201171875
lstm_history_ns Validation MSE: 39411000.0
lstm_history_ns Validation MAE: 3182.515869140625
-------------rnn_history-------------
rnn_history Validation Loss: 3148.002685546875
rnn_history Validation MSE: 37060056.0
rnn_history Validation MAE: 3148.46875
-------------rnn_history_ns-------------
rnn_history_ns Validation Loss: 3171.3798828125
rnn_history_ns Validation MSE: 38259400.0
rnn_history_ns Validation MAE: 3171.7568359375
-------------gru_history-------------
gru_history Validation Loss: 3191.7138671875
gru_history Validation MSE: 37720672.0
gru_history Validation MAE: 3192.104736328125
-------------gru_history_ns-------------
gru_history_ns Validation Loss: 3202.40380859375
gru_history_ns Validation MSE: 39184436.0
gru

In [16]:
lstm_save_path = f"./Models/lstm_model_stne_c1"
rnn_save_path = f"./Models/rnn_model_stne_c1"
gru_save_path = f"./Models/gru_model_stne_c1"

save_model(lstm_model, lstm_save_path, overwrite=True)
save_model(rnn_model, rnn_save_path, overwrite=True)
save_model(gru_model, gru_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Models/lstm_model_stne_c1\assets


INFO:tensorflow:Assets written to: ./Models/lstm_model_stne_c1\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_stne_c1\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_stne_c1\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_stne_c1\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_stne_c1\assets


In [17]:
St_NotEncode_Basic_data = pd.read_pickle("Basic_StandardScalar_final_data")
  
print(St_NotEncode_Basic_data.shape)

(52578, 9)


In [18]:
# Feature와 Label 분리하기 (풍속, 습도, 일조시간, 일사량, 전운량)
def Feature_Label(datafile) :
    X = datafile.iloc[:,[4,5]]
    y = datafile.iloc[:,-1]

    return X, y

print("StandardScaler Not Encode Data")
SB_X, SB_y = Feature_Label(St_NotEncode_Basic_data)
print(SNE_X.shape, SNE_y.shape)
SB_X_train, SB_X_test, SB_y_train, SB_y_test = train_test_split(SB_X, SB_y, test_size=0.2, random_state=10, shuffle=False)
print(SB_X_train.shape, SB_X_test.shape, SB_y_train.shape, SB_y_test.shape)
print("=======================================")


WINDOW_SIZE=3
BATCH_SIZE=28

StandardScaler Not Encode Data
(17422, 1) (17422,)
(42062, 2) (10516, 2) (42062,) (10516,)


In [19]:
def windowed_dataset(x, y, window_size, batch_size, shuffle) :
  ds_x = tf.data.Dataset.from_tensor_slices(x).window(window_size, stride=1, shift=1, drop_remainder=True) 
  ds_x = ds_x.flat_map(lambda x : x.batch(window_size))
  
  ds_y = tf.data.Dataset.from_tensor_slices(y[window_size:])
  
  ds = tf.data.Dataset.zip((ds_x, ds_y))
  
  if shuffle:
    ds = ds.shuffle(1000)
  
  return ds.batch(batch_size).prefetch(1)

In [20]:
train_data = windowed_dataset(SB_X_train, SB_y_train, WINDOW_SIZE, BATCH_SIZE, True)
train_data_ns = windowed_dataset(SB_X_train, SB_y_train, WINDOW_SIZE, BATCH_SIZE, False)
test_data = windowed_dataset(SB_X_test, SB_y_test, WINDOW_SIZE, BATCH_SIZE, False)

In [21]:
for data in train_data.take(1) :
  print(f'{data[0].shape}, {data[1].shape}')

(28, 3, 2), (28,)


In [22]:
lstm_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal',
         activation='relu',
         input_shape=[WINDOW_SIZE, 2]),
  LSTM(128, activation='tanh', return_sequences=True),  
  LSTM(64, activation='tanh', return_sequences=True),
  LSTM(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [23]:
rnn_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 2]),
  SimpleRNN(128, activation='tanh', return_sequences=True),  
  SimpleRNN(64, activation='tanh', return_sequences=True),
  SimpleRNN(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [24]:
gru_model = Sequential([
  Conv1D(filters=32, kernel_size=30,
         padding='causal', 
         activation='relu',
         input_shape=[WINDOW_SIZE, 2]),
  GRU(128, activation='tanh', return_sequences=True),  
  GRU(64, activation='tanh', return_sequences=True),
  GRU(32, activation='tanh'),
  Dense(8, activation='relu'),
  Dense(1)
])

In [25]:
loss = Huber() 
optimizer = Adam(learning_rate=0.001)
lstm_model.compile(loss=loss, optimizer=optimizer, metrics=['mse', 'mae'])
rnn_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])
gru_model.compile(loss=loss, optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), metrics=['mse', 'mae'])

In [26]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10)
       
es = EarlyStopping(monitor='val_loss', patience=20)

mc_lstm = ModelCheckpoint('base_multi_stne_lstm_weight_c2.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_lstm_ns = ModelCheckpoint('base_multi_stne_lstm_weight_ns_c2.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn = ModelCheckpoint('base_multi_stne_rnn_weight_c2.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_rnn_ns = ModelCheckpoint('base_multi_stne_rnn_weight_ns_c2.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru = ModelCheckpoint('base_multi_stne_gru_weight_c2.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)
mc_gru_ns = ModelCheckpoint('base_multi_stne_gru_weight_ns_c2.h5', monitor='val_loss', mode='auto', verbose=1, save_best_only=True)

In [27]:
lstm_history = lstm_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm])
lstm_history_ns = lstm_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_lstm_ns])

Epoch 1/500
   1499/Unknown - 35s 9ms/step - loss: 2.1048 - mse: 25.5503 - mae: 2.4631
Epoch 1: val_loss improved from inf to 2.15057, saving model to base_multi_stne_lstm_weight_c2.h5
1503/1503 [==============================] - 39s 11ms/step - loss: 2.1036 - mse: 25.5221 - mae: 2.4620 - val_loss: 2.1506 - val_mse: 22.2719 - val_mae: 2.5207 - lr: 0.0010
Epoch 2/500
1501/1503 [============================>.] - ETA: 0s - loss: 1.3452 - mse: 9.9153 - mae: 1.6847
Epoch 2: val_loss did not improve from 2.15057
1503/1503 [==============================] - 14s 9ms/step - loss: 1.3446 - mse: 9.9097 - mae: 1.6840 - val_loss: 2.1765 - val_mse: 22.3181 - val_mae: 2.5446 - lr: 0.0010
Epoch 3/500
1500/1503 [============================>.] - ETA: 0s - loss: 1.3126 - mse: 9.7369 - mae: 1.6471
Epoch 3: val_loss improved from 2.15057 to 2.02315, saving model to base_multi_stne_lstm_weight_c2.h5
1503/1503 [==============================] - 14s 9ms/step - loss: 1.3124 - mse: 9.7362 - mae: 1.6469 - val_l

In [28]:
rnn_history = rnn_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn])
rnn_history_ns = rnn_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_rnn_ns])

Epoch 1/500
   1496/Unknown - 20s 5ms/step - loss: 1.9174 - mse: 20.4215 - mae: 2.2698
Epoch 1: val_loss improved from inf to 2.09451, saving model to base_multi_stne_rnn_weight_c2.h5
1503/1503 [==============================] - 23s 7ms/step - loss: 1.9180 - mse: 20.4213 - mae: 2.2704 - val_loss: 2.0945 - val_mse: 20.9069 - val_mae: 2.4726 - lr: 0.0010
Epoch 2/500
1502/1503 [============================>.] - ETA: 0s - loss: 1.3711 - mse: 10.3292 - mae: 1.7087
Epoch 2: val_loss improved from 2.09451 to 2.04254, saving model to base_multi_stne_rnn_weight_c2.h5
1503/1503 [==============================] - 9s 6ms/step - loss: 1.3711 - mse: 10.3286 - mae: 1.7086 - val_loss: 2.0425 - val_mse: 20.8824 - val_mae: 2.4142 - lr: 0.0010
Epoch 3/500
1498/1503 [============================>.] - ETA: 0s - loss: 1.3429 - mse: 10.0331 - mae: 1.6798
Epoch 3: val_loss did not improve from 2.04254
1503/1503 [==============================] - 9s 6ms/step - loss: 1.3447 - mse: 10.0569 - mae: 1.6816 - val_lo

In [29]:
gru_history = gru_model.fit(train_data, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru])
gru_history_ns = gru_model.fit(train_data_ns, validation_data=test_data, epochs=500, callbacks=[reduce_lr, es, mc_gru_ns])

Epoch 1/500
   1501/Unknown - 18s 8ms/step - loss: 2.4044 - mse: 31.3898 - mae: 2.7680
Epoch 1: val_loss improved from inf to 2.19081, saving model to base_multi_stne_gru_weight_c2.h5
1503/1503 [==============================] - 20s 9ms/step - loss: 2.4038 - mse: 31.3812 - mae: 2.7674 - val_loss: 2.1908 - val_mse: 22.5529 - val_mae: 2.5103 - lr: 0.0010
Epoch 2/500
1497/1503 [============================>.] - ETA: 0s - loss: 1.3324 - mse: 9.8716 - mae: 1.6743
Epoch 2: val_loss improved from 2.19081 to 2.01207, saving model to base_multi_stne_gru_weight_c2.h5
1503/1503 [==============================] - 12s 8ms/step - loss: 1.3348 - mse: 9.9240 - mae: 1.6767 - val_loss: 2.0121 - val_mse: 20.1755 - val_mae: 2.3901 - lr: 0.0010
Epoch 3/500
1499/1503 [============================>.] - ETA: 0s - loss: 1.3235 - mse: 9.8502 - mae: 1.6589
Epoch 3: val_loss did not improve from 2.01207
1503/1503 [==============================] - 13s 9ms/step - loss: 1.3255 - mse: 9.8951 - mae: 1.6611 - val_loss

In [30]:
# 종합 결과

history_list = ["lstm_history", "rnn_history", "gru_history", "lstm_history_ns", "rnn_history_ns", "gru_history_ns"]
def result(historys) :
  for name, history in globals().items() :
    if name in history_list :
      print(f"-------------{name}-------------")
      val_loss = min(history.history['val_loss'])
      val_mse = min(history.history['val_mse'])
      val_mae = min(history.history['val_mae'])
      print(f"{name} Validation Loss:", val_loss)
      print(f"{name} Validation MSE:", val_mse)
      print(f"{name} Validation MAE:", val_mae)

result(history_list)

-------------lstm_history-------------
lstm_history Validation Loss: 1.9420496225357056
lstm_history Validation MSE: 19.080568313598633
lstm_history Validation MAE: 2.3100786209106445
-------------lstm_history_ns-------------
lstm_history_ns Validation Loss: 2.039152145385742
lstm_history_ns Validation MSE: 20.505569458007812
lstm_history_ns Validation MAE: 2.4030139446258545
-------------rnn_history-------------
rnn_history Validation Loss: 1.9659862518310547
rnn_history Validation MSE: 19.526901245117188
rnn_history Validation MAE: 2.3381295204162598
-------------rnn_history_ns-------------
rnn_history_ns Validation Loss: 2.0490355491638184
rnn_history_ns Validation MSE: 20.756359100341797
rnn_history_ns Validation MAE: 2.4220988750457764
-------------gru_history-------------
gru_history Validation Loss: 1.8992069959640503
gru_history Validation MSE: 18.856443405151367
gru_history Validation MAE: 2.262178659439087
-------------gru_history_ns-------------
gru_history_ns Validation Los

In [31]:
lstm_save_path = f"./Models/lstm_model_stne_base_c2"
rnn_save_path = f"./Models/rnn_model_stne_base_c2"
gru_save_path = f"./Models/gru_model_stne_base_c2"

save_model(lstm_model, lstm_save_path, overwrite=True)
save_model(rnn_model, rnn_save_path, overwrite=True)
save_model(gru_model, gru_save_path, overwrite=True)

INFO:tensorflow:Assets written to: ./Models/lstm_model_stne_base_c2\assets


INFO:tensorflow:Assets written to: ./Models/lstm_model_stne_base_c2\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_stne_base_c2\assets


INFO:tensorflow:Assets written to: ./Models/rnn_model_stne_base_c2\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_stne_base_c2\assets


INFO:tensorflow:Assets written to: ./Models/gru_model_stne_base_c2\assets
